<img src="Images/HSP2.png" />
This Jupyter Notebook Copyright 2017 by RESPEC, INC.  All rights reserved.

$\textbf{HSP}^{\textbf{2}}\ \text{and}\ \textbf{HSP2}\ $ Copyright 2017 by RESPEC INC. and released under this [License](LegalInformation/License.txt)

## Youtube Video Developed for HSP2 and this Tutorial

In [ ]:
from IPython.display import HTML
HTML('<iframe width="896" height="504" src="https://www.youtube.com/embed/aeLScKsP1Wk?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

## Required Python imports  and setup

In [ ]:
import os
import site
site.addsitedir(os.getcwd().rsplit('\\',1)[0] + '\\')  # adds your path to the HSP2 software.

import shutil
import numpy as np
import pandas as pd
pd.options.display.max_rows    = 18
pd.options.display.max_columns = 10
pd.options.display.float_format = '{:.2f}'.format  # display 2 digits after the decimal point

import HSP2
import HSP2tools

import qgrid
# Tell qgrid to automatically render all DataFrames and Series as qgrids.
qgrid.enable()
# Disable automatic display so we can display DataFrames in the normal way
# qgrid.disable()

import matplotlib.pyplot as m_plt
%matplotlib inline

HSP2tools.reset_tutorial()    # make a new copy of the tutorial's data
HSP2tools.versions()          # display version information below


## Importing UCI & WDM Files into HDF5<a id='section1'></a>

In [ ]:
uciname = os.path.join('TutorialData', 'test10.uci') 
wdmname = os.path.join('TutorialData', 'test.wdm')

unpackedhdfname = os.path.join('TutorialData', 'unpackedtutorial.h5')
hdfname = os.path.join('TutorialData', 'Tutorial.h5')

In [ ]:
HSP2tools.makeH5()
HSP2tools.readUCI(uciname, unpackedhdfname)
HSP2tools.ReadWDM(wdmname, unpackedhdfname)
!ptrepack {unpackedhdfname}  {hdfname}

## Run $\textbf{HSP}^\textbf{2}$

In [ ]:
HSP2.run??

In [ ]:
HSP2.run(hdfname, saveall=True)

In [ ]:
# Review simulated hydraulic state variables and fluxes from hdf5 file for Reach 1
tsMaster = pd.read_hdf(hdfname, '/RESULTS/RCHRES_R001/HYDR')
tsMaster

In [ ]:
# Create a Plot of RCHRES 1 Flow
m_plt.figure(figsize=(16,8))
m_plt.plot('RO', 'b--', data=tsMaster,   label='Master')

In [ ]:
# Creeate a DateFrame, Calculate Summart Stats, and Save for Later
columns = ['Master','Run1','Run3', 'Run4']
dfStats = pd.DataFrame(columns=columns)
dfStats.Master = tsMaster.RO.describe()

dfStats.Master

## There are Multiple ways we Can Change Parameters

###  Using HSP2Tools.Fetch and then HSP2Tools.replace

In [ ]:
replaceinfo, df = HSP2tools.fetch(hdfname, 'PERLND', 'PWATER', 'PARAMETERS')
replaceinfo, df

In [ ]:
# Check Value of LZSN
df.LZSN

In [ ]:
# Change and Replace using HSP2Tools.replace?? 
df.LZSN = 12
HSP2tools.replace(replaceinfo, df)

### Use Pandas to Read and Write

In [ ]:
datapath = '/PERLND/PWATER/PARAMETERS'
df2 = pd.read_hdf(hdfname, datapath)
df2.T

In [ ]:
# Write to HDF
df2.LZSN = 8
df2.to_hdf(hdfname, '/PERLND/PWATER/PARAMETERS', data_columns=True, format='table')

In [ ]:
# Verify Changed
df2 = pd.read_hdf(hdfname, datapath)
df2.LZSN

### Use Pandas and Qgrid Functionality to Update Parameters

In [ ]:
# List the dataframe and edit Qgrid directly (e.g., dbl click and change LZSN to 6) then use qgrid_widget.get_changed_df()
qgrid_widget = qgrid.QgridWidget(df=df2.T, show_toolbar=True)
qgrid_widget


In [ ]:
df3 = qgrid_widget.get_changed_df()
df3.T.to_hdf(hdfname, '/PERLND/PWATER/PARAMETERS', data_columns=True, format='table')

In [ ]:
# Verify Changed
df2 = pd.read_hdf(hdfname, datapath)
df2.LZSN

### Use HSPTools.csvReader to Update Paramters

In [ ]:
csvFile = os.path.join('TutorialData', 'perlnd.csv')
pd.read_csv(csvFile)

In [ ]:
# Use CSV to Update the HDF5 File
HSP2tools.csvReader(hdfname, csvFile, 'PERLND', 'PWATER')
# verify changed
var = ['INFILT','LZSN']
pd.read_hdf(hdfname, datapath)[var]

### Use HSPTools.DoE (Design of Experiment) function to Update Paramters

In [ ]:
# How Can we Cange Multiple Parameters and Track Changes
# Using the HSP2.DoE approach allows us to create new folders within HDF5 file to document 
# changes from base and store new results
data = [
 ['1', 'PERLND', 'P001', 'PWATER', 'INFILT',   0.075],
 ['1', 'PERLND', 'P001', 'PWATER', 'LZSN',         4],
 ['2', 'PERLND', 'P001', 'PWATER', 'INFILT',    0.30],
 ['2', 'PERLND', 'P001', 'PWATER', 'LZSN',      12.0]]

doe = pd.DataFrame(data, columns=['Run', 'Operation', 'Segment', 'Module', 'Parameter', 'Value'])
doe

In [ ]:
HSP2.run_DoE(hdfname, 'Sensitivity_LZSN_INFILT', doe, saveall=True)

In [ ]:
# Acquire and Calculate Stats on Run 1 & Run 2 and Compare to Master
tsRun_1  = pd.read_hdf(hdfname, 'Sensitivity_LZSN_INFILT/RESULTS/RUN1/RCHRES_R001/HYDR')
tsRun_2  = pd.read_hdf(hdfname, 'Sensitivity_LZSN_INFILT/RESULTS/RUN2/RCHRES_R001/HYDR')

dfStats.Run1 = tsRun_1.RO.describe()
dfStats.Run2 = tsRun_2.RO.describe()
dfStats

In [ ]:
m_plt.figure(figsize=(16,8))
m_plt.plot('RO', 'b--', data=tsMaster,   label='Master')
m_plt.plot('RO', 'r',   data=tsRun_1,    label='Run1')
m_plt.plot('RO', 'g-',   data=tsRun_2,  label='Run2')
m_plt.title('Flow at Reach 1')
m_plt.ylabel('Hourly Flow {CFS}')
m_plt.legend(loc='best') 

In [ ]:
# Resample for Monthly Flow
m_plt.figure(figsize=(16,8))
m_plt.plot('RO', 'b--', data=tsMaster.resample('M').mean(),   label='Master')
m_plt.plot('RO', 'r',   data=tsRun_1.resample('M').mean(),    label='Run1')
m_plt.plot('RO', 'g-',   data=tsRun_2.resample('M').mean(),  label='Run2')
m_plt.title('Flow at Reach 1')
m_plt.ylabel('Monthly Flow {CFS}')
m_plt.legend(loc='best') 

In [ ]:
# Use DoE approach to do a quick sensitivity analysis of LZSN
N = 20   # number of runs - generally large such as 1000 or 10,000
data = []
for r in range(1, N+1):
    run = str(r+10)
    data.append([run,'PERLND','P001','PWATER','LZSN', 1 + float(np.random.rand(1)) * 15])
   
doe = pd.DataFrame(data, columns=['Run', 'Operation', 'Segment', 'Module', 'Parameter', 'Value'])
doe

In [ ]:
HSP2.run_DoE(hdfname, 'P001_LZSN', doe)

In [ ]:
# Get paths to all the results for RCHRES
keys = ['P001_LZSN/RESULTS/RUN' + str(k+10) + '/RCHRES_R001/HYDR' for k in range(1,N+1)]
keys

In [ ]:
ts = pd.DataFrame()
for k in keys:
    colname = k[18:23]
    df = pd.read_hdf(hdfname, k)
    ts[colname] = df.ROVOL
    
ts.plot(label = 'simulated volume (ac-ft)',figsize = (18,6))

In [ ]:
df = ts.describe()
df.T

### Use a time series for the INFILT parameter

#### First, create a time series for INFILT and save it in the HDF5 file's /Timeseries directory.

###  Constant parameters in HSPF can be replaced by time series in $\textbf{HSP}^\textbf{2}$ 


Some HSPF parameters were optionally allow to vary in time using FLAG and MONTHLY tables. The other HSPF parameters were constants.
HSPF used the following algorithm to determine the parameter's value at any time in the simulation when it was allowed to vary:
+ First interpolate monthly table values to get daily values. 
+ The values at timesteps within each day are set to the day's daily value.

However, the HSPF Special Functions capability could be used to allow any HSPF parameter to vary over time.

This capability to vary any parameter over time is made more integral to $\textbf{HSP}^\textbf{2}$.

#### IMPLIMENTATION in $\textbf{HSP}^\textbf{2}$ 

Internally, $\textbf{HSP}^\textbf{2}$, creates a time series for each parameter over the entire simulation interval at the start of each activity's code. 

The rules for creating a time series are simple:
+ Whenever the EXT_SOURCES table directs a time series with the name of an HSPF parameter (in TMEMN) to the current OPSEQ operation and segment (TVOL and TVOLNO), then this time series will be used in place of the parameter. (Because this is different bahavior than HSP2, a logged message
alerts the user whenever this is done.)
+ Otherwise, if the flag and monthly table information used by HSPF to allow a parameter to vary over time is found in the $\textbf{HSP}^\textbf{2}$ tables, then the HSPF algorithm is used to create a time series over the entire simulation interval.
+ Otherwise, this was is constant parameter in HSPF. The constant value found in for the parameter from PARAMETERS table will be used to fill the array.

There is no additional performance hit to specify a time series for a parameter since all parameters are already treated as time series internally anyway.

Get the simulation's GLOBAL data to create a time index for this simulation. The new series must at least contain the simulations start, stop boundaries.

In [ ]:
gdata = pd.read_hdf(hdfname, '/CONTROL/GLOBAL')['Data']
gdata

The frequency does not need to be at any fixed value - HSP2 will resample (up or down) to make it correct.

In [ ]:
start = pd.to_datetime(gdata['sim_start'])
stop  = pd.to_datetime(gdata['sim_end'])

tindex = pd.date_range(start, stop, freq='h')
tindex

Just set some values.

In [ ]:
infilt = pd.Series(0.10, index=tindex)                 # set the value of 0.10 at each timestep
infilt['1976-03-01 01:00':'1976-11-01 05:00'] = 0.20   # overwrite for all datetimes in this interval (end points included)

infilt

Save to the HDF5 file

In [ ]:
infilt.to_hdf(hdfname, 'TIMESERIES/infilt')

####  Second, add a row to the EXT_SOURCES table to send this time series to PERLND INFILT for segment P001.

In [ ]:
ext = pd.read_hdf(hdfname, '/CONTROL/EXT_SOURCES')
nrows, ncols = ext.shape

nrows, ncols

In [ ]:
ext.loc[nrows] = ['*', 'infilt', '', '', 1.0, '', 'PERLND', 'INFILT', '', 'P001', '', 'Adding New series to control infilt']
ext.tail()

In [ ]:
ext.to_hdf(hdfname, '/CONTROL/EXT_SOURCES',  data_columns=True, format='table')

Now run the simulation

In [ ]:
HSP2tools.run_Tutorial(hdfname, saveall=True)
#HSP2.run(hdfname)

The infilt timeseries was found twice because it was made available to both SNOW and PWATER.

In [ ]:
# Acquire and Calculate Stats on Run 3 and Compare to Other Runs
tsRun_3  = pd.read_hdf(hdfname, '/RESULTS/RCHRES_R001/HYDR')

dfStats.Run3 = tsRun_3.RO.describe()
dfStats

In [ ]:
# Resample for Monthly Flow
m_plt.figure(figsize=(16,8))
m_plt.plot('RO', 'b--', data=tsMaster.resample('M').mean(),   label='Master')
m_plt.plot('RO', 'r',   data=tsRun_1.resample('M').mean(),    label='Run1')
m_plt.plot('RO', 'g',   data=tsRun_2.resample('M').mean(),    label='Run2')
m_plt.plot('RO', 'k',   data=tsRun_3.resample('M').mean(),    label='Run3')

m_plt.title('Flow at Reach 1')
m_plt.ylabel('Monthly Flow {CFS}')
m_plt.legend(loc='best') 

### MFACTOR and AFACTR, may replaced by a time series

+ The MFACTOR table column is found in the MASS_LINK and EXT_SOURCES tables
+ The AFACTR table column is found in the LINKS table

If an AFACTR or MFACTOR element in a table is a string that starts with an asterisk, then the string after the asterisk is the name of a timeseries to be found in the HDF5 TIMESERIES directory.  It is treated as a sparse array and padded appropriately (aggregation method SAME).

Otherwise, the AFACTOR or MFACTOR element should be a floating point number or string that can be converted into a floating point number. Internally, a timeseries is created with this value in every position.

So either way, any AFACTOR or MFACTOR is a timeseries for internal calculation. They are multiplied pointwise times the data timeseries specified by the table.


### Simulate a town growing and replacing  farm land during a simulation.

This scenario is a town (IMPLND segment I001) growing over time replacing farm land (PERLND segment P001). The total area of the two segments must remain constant. This example uses the HSPF test10 HDF5, tutorial.h5.

The total area of the two segments P001 and I001 is 9000 acres. 

The IMPLND area will increase linearly by 20% over the simulation period. That is the IMPLD segment will grow from 3000 to 3600 acres.
This requires the PERLND segment to shrink from 6000 to 5400 acres.

First, create a timeseries for IMPLND. Name it *implnd* and save in the HDF5 file.

This process uses the tindex computed in the last example.

In [ ]:
implnd = pd.Series(index=tindex)
implnd[tindex[0]] = 3000.
implnd[tindex[-1]] = 1.2 * 3000.
implnd = implnd.interpolate(how='time')

implnd.to_hdf(hdfname, 'TIMESERIES/implnd')

Create a timeseries for PERLND

Start with the original PERLND area and pointwise (in time) subtract the increase in the IMPLND segment.

In [ ]:
perlnd = 6000. - (implnd-3000.)         # Note: this is a full vector calculation

perlnd.to_hdf(hdfname, 'TIMESERIES/perlnd')

#### Modify the AFACTR entries in the LINKS table

It is necessary to indicate when and which time series will be used to replace a fixed AFACTR. 

In [ ]:
df = pd.read_hdf(hdfname, '/CONTROL/LINKS')
df

The PERLND AFACTR is at table index 0, the IMPLND  AFACTR at index 5.

Modify the AFACTR entries for these two rows and save back to the HDF5 file.

In [ ]:
df.loc[0, 'AFACTR'] = '*perlnd'
df.loc[5, 'AFACTR'] = '*implnd'
df.AFACTR = df.AFACTR.astype(str)   # previously all entries were floats, so Pandas made this a float typed column.
df.to_hdf(hdfname, '/CONTROL/LINKS',  data_columns=True, format='table')
qgrid.disable()
df

Now run the simulation and look for the message displayed whenever AFACTR is replaced by a time series.

In [ ]:
HSP2tools.run_Tutorial(hdfname, saveall = 'True')
#HSP2.run(hdfname)

In [ ]:
# Acquire and Calculate Stats on Run 4 and Compare to Other Runs
tsRun_4  = pd.read_hdf(hdfname, '/RESULTS/RCHRES_R001/HYDR')

dfStats.Run4 = tsRun_4.RO.describe()
qgrid.show_grid(dfStats)

In [ ]:
# Resample for Weekly Flow
m_plt.figure(figsize=(16,8))
m_plt.plot('RO', 'b--', data=tsMaster.resample('7D').mean(),   label='Master')
m_plt.plot('RO', 'r',   data=tsRun_1.resample('7D').mean(),    label='Run1')
m_plt.plot('RO', 'g',   data=tsRun_2.resample('7D').mean(),    label='Run2')
m_plt.plot('RO', 'k',   data=tsRun_3.resample('7D').mean(),    label='Run3')
m_plt.plot('RO', 'c',   data=tsRun_4.resample('7D').mean(),    label='Run4')

m_plt.title('Flow at Reach 1')
m_plt.ylabel('Monthly Flow {CFS}')
m_plt.legend(loc='best') 